In [ ]:
import os
num_gpus=5 #How many gpus to use concurrently

scaffold_dir="../../scaffolds/scaffolds_V5/"
separate_scaffold_folders=True # Is every scaffold pdb is in its own folder?

prefix="CD4-dom2-sample44-noise-40x" # Name of output folder
target_pdb="target_pdb/c4-dom2-1cdh.pdb" # Target for binder
hotspots='ppi.hotspot_res=[A170,A1130,A165,A157,A135]' # Aminoacids which should be close to binder

num_of_diffusions=40 # Number of RF diffusions to do per scaffold
num_seqs=50 # How many MPNN sequences to generate per RF diffusion
num_filtered_seqs=50 # How many of the MPNN generated sequences per RF diffusion to keep for AF2

# RFdiffusion parameters
diff_steps = 50 # Number of diffuser steps (diffuser.T)
guiding_potentials=f'["type:binder_ROG,weight:3","type:interface_ncontacts","type:binder_distance_ReLU"]'
guide_scale=f'2'
guide_decay=f'"quadratic"'
noise_scale_ca='0.5'
noise_scale_frame='0.5'

# Af2 Mpnn parameters
num_recycles=3 # AF2 recycles
sampling_temp=0.0001 # ProteinMPNN sampling temperature

if "[A" in hotspots:
    rfdiffusion_config.append(hotspots)
rfdiffusion_argument_string= "+".join(rfdiffusion_config)

def get_subdirectories(folder_path):
    subdirectories = []
    for entry in os.scandir(folder_path):
        if entry.is_dir():
            subdirectories.append(entry.path)
    return subdirectories


if separate_scaffold_folders:
    scaffold_list=get_subdirectories(scaffold_dir)
    scaffolds=len(scaffold_list)
    print(scaffold_list)
print(f"JOB: {prefix}")
print(f"TARGET: {target_pdb}")
print(f"HOTSPOTS: {hotspots}")

# Checks if the output folder already exists
def directory_exists(directory):
    return os.path.isdir(directory)
directory = f"output/{prefix}"
if directory_exists(directory):
    print("""\033[1;31;40m#####################
PREFIX ALREADY EXISTS
#####################""")
else:
    print("Prefix does not exist yet")

# Counts the number of gpus already in use
gpus_in_use = !squeue | grep -c 'gpu.*compute\|compute.*gpu'
print(f"\033[0mNumber of gpus already in use: {gpus_in_use[0]}/50", end="")

## Runs slurm array script

In [ ]:
import subprocess
os.makedirs(directory, exist_ok=True)
if separate_scaffold_folders:
    if num_gpus%scaffolds==0:   #If number of gpus is divisible by the number of scaffolds, make it so that each scaffold has its own gpu(s)
        gpus_per_scaffold = num_gpus//scaffolds
        diffusions_per_gpu = num_of_diffusions//gpus_per_scaffold
        for i in range(scaffolds):
            # Setup arguments
            scaffold_dir=scaffold_list[i]
            script_arguments=f"{diffusions_per_gpu} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {rfdiffusion_argument_string}"
            bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{gpus_per_scaffold}"
            
            command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

            # Run the array bash script
            print(f"Running {command}")
            subprocess.run(command, shell=True)
    else:
        num_per_gpu=num_of_diffusions//num_gpus
        scaffold_string = ','.join(scaffold_list)   #Converts list of scaffold directories into a string for parsing in shell script
        script_arguments=f"{num_per_gpu} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_string} {prefix} {target_pdb} {rfdiffusion_argument_string}"
        bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{num_gpus}"
            
        command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

        # Run the array bash script
        print(f"Running {command}")
        subprocess.run(command, shell=True)

else:
    script_arguments=f"{num_of_diffusions} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {rfdiffusion_argument_string}"
    bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{num_gpus}"
    
    command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

    # Run the array bash script
    print(f"Running {command}")
    subprocess.run(command, shell=True)

# print(f"command: {command}")

In [ ]:
!squeue --me